# Clean Data
We mainly have three portions to prepare and clean our data for further analysis.
* Clean TLC monthly data : NYC TLC provide Yellow Taxi, Green Taxi, and FHV riding records for each month.
* Clean TLC location data : NYC TLC provide the definition of location id which is recorded on each month data.
* Find airports near by NYC with Google API

# Clean TLC monthly Data

In [2]:
#We don't put original raw data which is too huge on github, so you need to download the files from TLC to run this.
#Need to use this code to clean Yellow, Green, and FHV data of each month in 2018
#In the end, we will have 12 csv files for each month. Each file is combined Taxi and FHV and did aggregation.
#Then, we need to merge these 12 aggregative and clean data into one file. (using combine_several_datasets.ipynb code)

In [21]:
import pandas as pd 

## Yellow Taxi Data Clean

Use parse_dates to turn time variable columns into datetime type

In [22]:
yellow_Jan = pd.read_csv("Data/Jan/yellow_tripdata_2018-01.csv",
                         parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
yellow_Jan.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.00,0.0,0.3,8.30
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.00,0.0,0.3,34.80
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55


Only choose two columns from data because FHV data sets only have these two columns

In [23]:
yellow_Jan = yellow_Jan[["tpep_pickup_datetime","PULocationID"]]
yellow_Jan = yellow_Jan.rename(columns = {"tpep_pickup_datetime":"pickup_time","PULocationID":"pickup_ID"})
yellow_Jan.head()


,pickup_time,pickup_ID
0,2018-01-01 00:21:05,41
1,2018-01-01 00:44:55,239
2,2018-01-01 00:08:26,262
3,2018-01-01 00:20:22,140
4,2018-01-01 00:09:18,246


Create a new column "month" to record month value and then only keep the month. 

In [24]:
yellow_Jan["month"] = yellow_Jan.pickup_time.dt.month
yellow_Jan = yellow_Jan.loc[(yellow_Jan.month ==1)]
yellow_Jan.head()

,pickup_time,pickup_ID,month
0,2018-01-01 00:21:05,41,1
1,2018-01-01 00:44:55,239,1
2,2018-01-01 00:08:26,262,1
3,2018-01-01 00:20:22,140,1
4,2018-01-01 00:09:18,246,1


In [25]:
yellow_Jan.dropna(inplace=True) # Delete the NaN row for pickup_ID


In [26]:
yellow_Jan.head()

,pickup_time,pickup_ID,month
0,2018-01-01 00:21:05,41,1
1,2018-01-01 00:44:55,239,1
2,2018-01-01 00:08:26,262,1
3,2018-01-01 00:20:22,140,1
4,2018-01-01 00:09:18,246,1


In [28]:
data = yellow_Jan.groupby(["pickup_ID"]) #Using groupby for pickup_ID
count_num = list(data.count().month) #Do calculate how much trip for each pickup_ID
pick_id = list(data.head(1).pickup_ID.sort_values()) #Sort pickup_ID value 
new_yellow_Jan = pd.DataFrame({"Pickup_ID":pick_id,"month":1,"count":count_num})
new_yellow_Jan.head()

,Pickup_ID,month,count
0,1,1,571
1,2,1,4
2,3,1,37
3,4,1,19656
4,5,1,2


## Green Taxi Data Clean

Follow Yellow Taxi Code to do the same thing

In [29]:
green_Jan = pd.read_csv("Data/Jan/green_tripdata_2018-01.csv",
                         parse_dates=['lpep_pickup_datetime', 'lpep_dropoff_datetime'])
green_Jan.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2018-01-01 00:18:50,2018-01-01 00:24:39,N,1,236,236,5,0.70,6.0,0.5,0.5,0.0,0.0,NaN,0.3,7.3,2,1.0
1,2,2018-01-01 00:30:26,2018-01-01 00:46:42,N,1,43,42,5,3.50,14.5,0.5,0.5,0.0,0.0,NaN,0.3,15.8,2,1.0
2,2,2018-01-01 00:07:25,2018-01-01 00:19:45,N,1,74,152,1,2.14,10.0,0.5,0.5,0.0,0.0,NaN,0.3,11.3,2,1.0
3,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,-3.0,-0.5,-0.5,0.0,0.0,NaN,-0.3,-4.3,3,1.0
4,2,2018-01-01 00:32:40,2018-01-01 00:33:41,N,1,255,255,1,0.03,3.0,0.5,0.5,0.0,0.0,NaN,0.3,4.3,2,1.0


In [30]:
green_Jan = green_Jan[["lpep_pickup_datetime","PULocationID"]]
green_Jan = green_Jan.rename(columns = {"lpep_pickup_datetime":"pickup_time","PULocationID":"pickup_ID"})
green_Jan.head()

,pickup_time,pickup_ID
0,2018-01-01 00:18:50,236
1,2018-01-01 00:30:26,43
2,2018-01-01 00:07:25,74
3,2018-01-01 00:32:40,255
4,2018-01-01 00:32:40,255


In [31]:
green_Jan["month"] = green_Jan.pickup_time.dt.month
green_Jan = green_Jan.loc[(green_Jan.month ==1)] 
green_Jan.head()

,pickup_time,pickup_ID,month
0,2018-01-01 00:18:50,236,1
1,2018-01-01 00:30:26,43,1
2,2018-01-01 00:07:25,74,1
3,2018-01-01 00:32:40,255,1
4,2018-01-01 00:32:40,255,1


In [32]:
green_Jan.dropna(inplace=True)

In [33]:
data_green = green_Jan.groupby(["pickup_ID"])
count_num_green = list(data_green.count().month)
pick_id = list(data_green.head(1).pickup_ID.sort_values())
new_green_Jan = pd.DataFrame({"Pickup_ID":pick_id,"month":1,"count":count_num_green}) 
new_green_Jan.head()

,Pickup_ID,month,count
0,1,1,13
1,3,1,395
2,6,1,5
3,7,1,39137
4,8,1,10


## Combine Yellow and Green Taxi

In [34]:
#merge two data sets by outer method
taxi_Jan = pd.merge(new_yellow_Jan,new_green_Jan,on="Pickup_ID",how = "outer")
taxi_Jan.fillna({"count_x":0,"count_y":0,"month_x":1},inplace=True) #Fill 0 for NaN column on count and month value on month
taxi_Jan["count"] = taxi_Jan.count_x + taxi_Jan.count_y #Sum two counts to total count
taxi_Jan = taxi_Jan.rename(columns = {"month_x":"month"})
taxi_Jan = taxi_Jan[["Pickup_ID","month","count"]]
taxi_Jan["category"] = "Taxi"
taxi_Jan.head()

,Pickup_ID,month,count,category
0,1,1.0,584.0,Taxi
1,2,1.0,4.0,Taxi
2,3,1.0,432.0,Taxi
3,4,1.0,19656.0,Taxi
4,5,1.0,2.0,Taxi


## FHV Data Clean

In [35]:
FHV_Jan = pd.read_csv("Data/Jan/fhv_tripdata_2018-01.csv",
                         parse_dates=['Pickup_DateTime', 'DropOff_datetime'])
FHV_Jan.head()

,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num
0,2018-01-30 21:06:50,2018-01-30 21:15:34,56.0,129.0,NaN,B02884,NaN
1,2018-01-30 21:20:36,2018-01-30 21:35:29,129.0,112.0,NaN,B02884,NaN
2,2018-01-30 21:04:45,2018-01-30 21:16:34,47.0,42.0,NaN,B02884,NaN
3,2018-01-30 21:11:51,2018-01-30 21:40:35,49.0,131.0,NaN,B02884,NaN
4,2018-01-30 21:43:39,2018-01-30 21:49:59,98.0,121.0,NaN,B02884,NaN


In [36]:
FHV_Jan = FHV_Jan[["Pickup_DateTime","PUlocationID"]]
FHV_Jan = FHV_Jan.rename(columns = {"Pickup_DateTime":"pickup_time","PUlocationID":"pickup_ID"})
FHV_Jan.head()

,pickup_time,pickup_ID
0,2018-01-30 21:06:50,56.0
1,2018-01-30 21:20:36,129.0
2,2018-01-30 21:04:45,47.0
3,2018-01-30 21:11:51,49.0
4,2018-01-30 21:43:39,98.0


In [37]:
FHV_Jan["month"] = FHV_Jan.pickup_time.dt.month
FHV_Jan = FHV_Jan.loc[(FHV_Jan.month ==1) & (FHV_Jan.pickup_ID >0)]
FHV_Jan.head()

,pickup_time,pickup_ID,month
0,2018-01-30 21:06:50,56.0,1
1,2018-01-30 21:20:36,129.0,1
2,2018-01-30 21:04:45,47.0,1
3,2018-01-30 21:11:51,49.0,1
4,2018-01-30 21:43:39,98.0,1


In [38]:
FHV_Jan.dropna(inplace=True)

In [39]:
data_FHV = FHV_Jan.groupby(["pickup_ID"])
count_num_FHV = list(data_FHV.count().month)
pick_id = list(data_FHV.head(1).pickup_ID.sort_values())
new_FHV_Jan = pd.DataFrame({"Pickup_ID":pick_id,"month":1,"count":count_num_FHV,"category":"FHV"})
new_FHV_Jan.head()

,Pickup_ID,month,count,category
0,1.0,1,3309,FHV
1,2.0,1,26,FHV
2,3.0,1,23060,FHV
3,4.0,1,77546,FHV
4,5.0,1,2575,FHV


## Combine Taxi and FHV data 

In [40]:
taxi_FHV_Jan = pd.concat([taxi_Jan, new_FHV_Jan], axis=0)
taxi_FHV_Jan.reset_index(drop=True,inplace=True)
taxi_FHV_Jan.head()

,Pickup_ID,month,count,category
0,1.0,1.0,584.0,Taxi
1,2.0,1.0,4.0,Taxi
2,3.0,1.0,432.0,Taxi
3,4.0,1.0,19656.0,Taxi
4,5.0,1.0,2.0,Taxi


## Export combine data of Taxi and FHV data for each month

In [20]:
taxi_FHV_Jan.to_csv("All_Jan.csv",index=False)

# Combine each month clean data sets into one data set

In [ ]:
#You need to use the data file which is clean for each month and store them to run this to combine several months.
data_Jan = pd.read_csv("Data/All_Jan_out_Aug.csv")
data_Feb = pd.read_csv("Data/All_July_Sep.csv")
data_Jan.head()

In [ ]:
#Use the concat function to do stack for several data sets
data_Jan_Mar = pd.concat([data_Jan,data_Feb], axis=0)
data_Jan_Mar.reset_index(drop=True,inplace=True)
data_Jan_Mar.head()

In [ ]:
data_Jan_Mar.to_csv("All_TLC_Trip.csv",index=False)

# TLC Location ID Clean
There are around 260 location ID from TLC, so we need to find the lat. and lng. for each location. Then, we can use these information to plot heatmap for our analysis.

In [ ]:
import pandas as pd
import json
import requests
import os
import pprint

# Import API key
from api_keys import gkey
#Import original file from TLC
loc_file = pd.read_csv("taxi_zones.csv")
#Create two new columns to store lat. and lng.
loc_file["Lat"] =""
loc_file["Lng"] =""

loc_file.head(5)

In [ ]:
#Using Google API to find lat. and lng. of each location.
base_url ='https://maps.googleapis.com/maps/api/geocode/json'

locs = len(loc_file)
# get latlng for each zone
for index, row  in loc_file.iterrows():
    city = row['zone']+ "," + row["borough"] +"," +"NYC"
    params = {"address": city, "key": gkey}

    #response = requests.get(base_url,params= params)
    #print (response.url)
    get_data = requests.get(base_url,params= params).json()
    
    if get_data["status"] == 'OK':
        lat = get_data["results"][0]["geometry"]["location"]["lat"]
        lng = get_data["results"][0]["geometry"]["location"]["lng"]
    
        loc_file.loc[index,"Lat"] = lat
        loc_file.loc[index,'Lng'] =lng
        print (f" City {row['zone']} Lat {lat} Lng {lng}")
    else:
        print (f" City {row['zone']} Not Found")
#pprint(get_data)
#print(json.dumps(get_data, indent=4, sort_keys=True)
print("done")

In [ ]:
loc_file.to_csv("TLC_Locations.csv", index = None, header=True) 

# Lat and long of Airports near by NYC

In [ ]:
import requests
import json
import pandas as pd
# Google developer API key
from config import gkey

# Target city
target_apt = "Airport, New York"

# Build the endpoint URL
target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_apt, gkey)

In [ ]:
# Run a request to endpoint and convert result to json
geo_data = requests.get(target_url).json()

# Print the json
print(geo_data)

In [ ]:
# geocoordinates
target_coordinates = "40.6413111, -73.77813909999999"
target_search = "Airport"
target_radius = 80000
target_type = "airport"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

In [ ]:
# convert response to json
apt_data = response.json()

# Print the json (pretty printed)
print(json.dumps(apt_data, indent=4, sort_keys=True))

In [ ]:
name_apt=[]
lat=[]
lng=[]
i=0
for i in range(len(apt_data["results"])):
    name_apt.append(apt_data["results"][i]["name"])
  
    lat.append( apt_data["results"][i]["geometry"]["location"]["lat"])
    lng.append( apt_data["results"][i]["geometry"]["location"]["lng"])
    i+=1
print(name_apt)

In [ ]:
apt_dict={"Airport":name_apt,
             "Latitude":lat,
             "Longitude":lng}
apt_df=pd.DataFrame(apt_dict)
apt_df.head(7)

In [ ]:
apt_df.to_csv("Data/airport.csv",index=None, header=True)